In [11]:
import os
import sys

import base64
import zlib
import bz2
import lzma
import zstd

from main import compress, decompress, Compressed

from llama_cpp import Llama

In [12]:
# choose a model

# throw metal warnings into the void
stderr_backup = sys.stderr
sys.stderr = open(os.devnull, 'w')

try: 
    llm = Llama.from_pretrained(
        repo_id="QuantFactory/SmolLM2-360M-GGUF",
        filename="*Q4_0.gguf",
        verbose=False,
        logits_all=True,
        n_gpu_layers=-1,
        n_ctx=32768,
    )
finally:
    sys.stderr.close()
    sys.stderr = stderr_backup

In [13]:
# choose input to compress
input_text = "test"

In [14]:
# compress text
compressed = compress(llm, input_text)
print(compressed.to_bytes())

b'\x00\x00\x00\x01\x00\x00\x00\x18\xff\xf0\xc8'


In [15]:
# compare results
original_bytes = input_text.encode("utf-8")
original_size = len(original_bytes)

# calc llm compression ratio
compressed_size = len(compressed.to_bytes())
compression_ratio = original_size / compressed_size

# calc compression algorithms
gzip_size = len(zlib.compress(original_bytes, level=9))
bz2_size = len(bz2.compress(original_bytes, compresslevel=9))
lzma_size = len(lzma.compress(original_bytes, preset=9))
zstd_size = len(zstd.compress(original_bytes, 22))

print(f"Encoded: {base64.b64encode(compressed.to_bytes()).decode('ascii')}")
print("\nCompression Results:")
print(f"  Original:        {original_size:>6} bytes")
print(
    f"  LLM compressed:  {compressed_size:>6} bytes ({compression_ratio:.2f}x)"
)
print(
    f"  ZSTD (level 22):  {zstd_size:>6} bytes ({original_size / zstd_size:.2f}x)"
)
print(
    f"  GZIP (level 9):  {gzip_size:>6} bytes ({original_size / gzip_size:.2f}x)"
)
print(
    f"  BZ2 (level 9):   {bz2_size:>6} bytes ({original_size / bz2_size:.2f}x)"
)
print(
    f"  LZMA (level 9):  {lzma_size:>6} bytes ({original_size / lzma_size:.2f}x)"
)

Encoded: AAAAAQAAABj/8Mg=

Compression Results:
  Original:             4 bytes
  LLM compressed:      11 bytes (0.36x)
  ZSTD (level 22):      13 bytes (0.31x)
  GZIP (level 9):      12 bytes (0.33x)
  BZ2 (level 9):       40 bytes (0.10x)
  LZMA (level 9):      60 bytes (0.07x)


In [17]:
# confirm decompression
to_decompress = Compressed.from_bytes(compressed.to_bytes())
decompressed = decompress(llm, to_decompress)
assert decompressed == input_text, "Decompression failed!"
print("\nDecompression successful!")

decompressed


Decompression successful!


'test'